In [1]:
%matplotlib widget
import numpy as np
import seaborn as sns
import pandas as pd
from functools import reduce
from tqdm import tqdm
from collections import defaultdict
import matplotlib.pyplot as plt
import sys
import os


In [2]:
def load_csv(path):
    return pd.read_csv(path)

In [4]:
rstats_path = ""
rand_rstats_path = ""

In [ ]:
rstats = load_csv(rstats_path)
rand_rstats = load_csv(rand_rstats_path)

## Uncomment to restrict maximum radius in the plots
# rstats = rstats[rstats["Radius (r)"] <= 102]
# rand_rstats = rand_rstats[rand_rstats["Radius (r)"] <= 102]

In [ ]:
f, axes = plt.subplots(3)

# K
sns.lineplot(data=rstats, x="Radius (r)", y="K(r)", ax=axes[1, 0])
sns.lineplot(data=rand_rstats, x="Radius (r)", y="K(r)", ax=axes[1, 0], alpha=0, errorbar=("pi", 95))
# L
sns.lineplot(data=rstats, x="Radius (r)", y="L(r)", ax=axes[1, 1])
sns.lineplot(data=rand_rstats, x="Radius (r)", y="L(r)", ax=axes[1, 1], alpha=0, errorbar=("pi", 95))
axes[1, 1].plot([0, 400], [0, 400], linewidth=1, linestyle="dotted", color="#aaa") # Expected line
# H
sns.lineplot(data=rstats, x="Radius (r)", y="H(r)", ax=axes[1, 2])
sns.lineplot(data=rand_rstats, x="Radius (r)", y="H(r)", ax=axes[1, 2], alpha=0, errorbar=("pi", 95))
axes[1, 2].plot([0, 400], [0, 0], linewidth=1, linestyle="dotted", color="#aaa") # Expected line

plt.show()